## Web of Science Data

In [3]:
import pandas as pd

### Clean and Filter Affiliations

In [43]:
def specific_college_csv(affiliation, short_name, keyword_list, all_keywords = False):
    # Read CSV data into pandas dataframes
    big_affiliations_df = pd.DataFrame()
    big_articles_df = pd.DataFrame()
    big_authors_df = pd.DataFrame()
    big_keywords_df = pd.DataFrame()
    for i in range(1,26):
        temp_affiliations_df = pd.read_csv(f"C:/Users/xieya/Desktop/Spring 2022/VIP VXG/Tableau mini project/Non-GT Data/affiliations/2020_affiliations_{i}.csv")
        articles_df = pd.read_csv(f"C:/Users/xieya/Desktop/Spring 2022/VIP VXG/Tableau mini project/Non-GT Data/articles/2020_articles_{i}.csv", index_col=False)
        authors_df = pd.read_csv(f"C:/Users/xieya/Desktop/Spring 2022/VIP VXG/Tableau mini project/Non-GT Data/authors/2020_authors_{i}.csv")
        keywords_df = pd.read_csv(f"C:/Users/xieya/Desktop/Spring 2022/VIP VXG/Tableau mini project/Non-GT Data/keywords/2020_keywords_{i}.csv")
        
        affiliations_df = temp_affiliations_df[temp_affiliations_df.organization == affiliation]
        # Redefine affiliations_df to include coauthors of <affiliation> people, even if they aren't a <affiliation> employee
        affiliations_df = temp_affiliations_df[temp_affiliations_df["docID"].isin(affiliations_df["docID"])]

        # Drop duplicate affiliations
        affiliations_df = affiliations_df.drop_duplicates()
        affiliations_df = affiliations_df.dropna()

        if not all_keywords:
            # Limits keywords only from <keyword_list>
            keywords_list_df = keywords_df[(keywords_df["keyword"].str.lower()).isin(keyword_list)]
            # # Limits affiliations only for those articles in <keyword_list>
            affiliations_df = affiliations_df[affiliations_df["docID"].isin(keywords_list_df["docID"])]
            # # Limits articles only for those articles in <keyword_list>
            articles_df = articles_df[articles_df["docID"].isin(keywords_list_df["docID"])]

        # Limit articles to those with author affiliated with <affiliation>
        articles_df = articles_df[articles_df["docID"].isin(affiliations_df["docID"])]

        # Drop na values
        articles_df = articles_df.dropna(subset=["title"])
        
        # Limit list of authors to those with affiliation to <affiliation>
        authors_df = authors_df[authors_df["docID"].isin(affiliations_df["docID"])]
        # Limits authors only for those articles in <keyword_list>
        authors_df = authors_df[authors_df["docID"].isin(keywords_list_df["docID"])]
        # Create dummy columns
        authors_df["dummy_given_name"] = authors_df["wosFirstName"]
        authors_df["dummy_family_name"] = authors_df["wosLastName"]
        authors_df["dummy_first_initial"] = authors_df["wosFirstName"]
        # Drop rows with no first or last name
        authors_df = authors_df.dropna(subset=["wosFirstName", "wosLastName"])
        authors_df = authors_df.dropna(subset=["wosFullName"])

        ## Standardize all dummy names
        # Take first part of name and make lowercase
        authors_df["dummy_given_name"] = \
            authors_df["dummy_given_name"].apply(lambda x : x.split(" ")[0].split("-")[0].lower())

        authors_df["dummy_family_name"] = \
            authors_df["dummy_family_name"].apply(lambda x : x.split(" ")[0].split("-")[0].lower())

        # Remove non-alphabetic characters
        authors_df["dummy_given_name"] = \
            authors_df["dummy_given_name"].apply(lambda x : "".join(filter(str.isalpha, x)))

        authors_df["dummy_family_name"] = \
            authors_df["dummy_family_name"].apply(lambda x : "".join(filter(str.isalpha, x)))

        # Create first initial
        authors_df["dummy_first_initial"] = \
            authors_df["dummy_first_initial"].apply(lambda x : x[0].lower())
        # Limit keywords to those from <affiliation> papers
        keywords_df = keywords_df[keywords_df["docID"].isin(affiliations_df["docID"])]
        keywords_df = keywords_df[keywords_df["docID"].isin(keywords_list_df["docID"])]
        keywords_df["keyword"] = keywords_df["keyword"].str.lower() #make everything lower case

        # Concat all files to big dataframe
        big_affiliations_df = pd.concat([big_affiliations_df,affiliations_df], axis=0)
        big_articles_df = pd.concat([big_articles_df,articles_df], axis=0)
        big_authors_df = pd.concat([big_authors_df,authors_df], axis=0)
        big_keywords_df = pd.concat([big_keywords_df,keywords_df], axis=0)

    big_affiliations_df.to_csv(f"C:/Users/xieya/Desktop/Spring 2022/zS2022 EVPR/Web of Science/2020_{short_name}_affiliations.csv", index=False)
    big_articles_df.to_csv(f"C:/Users/xieya/Desktop/Spring 2022/zS2022 EVPR/Web of Science/2020_{short_name}_articles.csv", index=False)
    big_authors_df.to_csv(f"C:/Users/xieya/Desktop/Spring 2022/zS2022 EVPR/Web of Science/2020_{short_name}_authors.csv", index=False)
    big_keywords_df.to_csv(f"C:/Users/xieya/Desktop/Spring 2022/zS2022 EVPR/Web of Science/2020_{short_name}_keywords.csv", index=False)

In [44]:
info_science_keywords = ["machine learning", "artificial intelligence", "cybersecurity", "cyber security", "climate security","environmental security"]
# school = "North Carolina A&T State University"
# short = "NCA&T"

schools_dict = {"North Carolina A&T State University":"NCA&T", "Prairie View A&M University":"PVA&M", "Florida A&M University": "FA&M", 
"Alabama A&M University":"AA&M", "Norfolk State Univ":"NSU"}
for index, (school, short) in enumerate(schools_dict.items()):
    specific_college_csv(affiliation = school, short_name = short, keyword_list = info_science_keywords)

### Combining all colleges

In [45]:
master_affiliations_df = pd.DataFrame()
master_articles_df = pd.DataFrame()
master_authors_df = pd.DataFrame()
master_keywords_df = pd.DataFrame()

for index, (school, short) in enumerate(schools_dict.items()):
    col_affiliations_df = pd.read_csv(f"C:/Users/xieya/Desktop/Spring 2022/zS2022 EVPR/Web of Science/2020_{short}_affiliations.csv")
    col_articles_df = pd.read_csv(f"C:/Users/xieya/Desktop/Spring 2022/zS2022 EVPR/Web of Science/2020_{short}_articles.csv", index_col=False)
    col_authors_df = pd.read_csv(f"C:/Users/xieya/Desktop/Spring 2022/zS2022 EVPR/Web of Science/2020_{short}_authors.csv")
    col_keywords_df = pd.read_csv(f"C:/Users/xieya/Desktop/Spring 2022/zS2022 EVPR/Web of Science/2020_{short}_keywords.csv")

    master_affiliations_df = pd.concat([master_affiliations_df,col_affiliations_df], axis=0)
    master_articles_df = pd.concat([master_articles_df,col_articles_df], axis=0)
    master_authors_df = pd.concat([master_authors_df,col_authors_df], axis=0)
    master_keywords_df = pd.concat([master_keywords_df,col_keywords_df], axis=0)

master_affiliations_df.to_csv(f"C:/Users/xieya/Desktop/Spring 2022/zS2022 EVPR/Web of Science/master_2020_affiliations.csv", index=False)
master_articles_df.to_csv(f"C:/Users/xieya/Desktop/Spring 2022/zS2022 EVPR/Web of Science/master_2020_articles.csv", index=False)
master_authors_df.to_csv(f"C:/Users/xieya/Desktop/Spring 2022/zS2022 EVPR/Web of Science/master_2020_authors.csv", index=False)
master_keywords_df.to_csv(f"C:/Users/xieya/Desktop/Spring 2022/zS2022 EVPR/Web of Science/master_2020_keywords.csv", index=False)